### ウェザーデータを獲得する

In [7]:
import requests

# 你的 OpenWeather API Key
API_KEY = "4c90f7b876a2f86509856f59409ffc33"

# 要转换的地址
address = "Tokyo, Japan"

# 调用地理编码接口
geo_url = "http://api.openweathermap.org/geo/1.0/direct"
params = {
    "q": address,
    "limit": 1,        # 最多返回 1 条匹配
    "appid": API_KEY
}

resp = requests.get(geo_url, params=params)
resp.raise_for_status()       # 有错误会抛异常
data = resp.json()

if data:
    lat1 = data[0]["lat"]
    lon1 = data[0]["lon"]
    print(f"经度：{lon}, 纬度：{lat}")
else:
    print("没有找到对应的经纬度，请检查地址拼写或调整查询范围。")


经度：139.7638947, 纬度：35.6768601


In [9]:
import requests
import datetime

API_KEY = "4c90f7b876a2f86509856f59409ffc33"
lat = 35.6768601
lon = 139.7638947

# 以2024-04-01为例，获取当天0点的UNIX时间戳
dt = int(datetime.datetime(2024, 4, 1, 0, 0).timestamp())

url = "https://history.openweathermap.org/data/2.5/history/city"
params = {
    "lat": lat,
    "lon": lon,
    "type": "hour",
    "start": dt,
    "end": dt + 24*3600,
    "appid": API_KEY,
    "units": "metric",
    "lang": "zh_cn"
}

resp = requests.get(url, params=params)
resp.raise_for_status()
data = resp.json()
print(data)

HTTPError: 401 Client Error: Unauthorized for url: https://history.openweathermap.org/data/2.5/history/city?lat=35.6768601&lon=139.7638947&type=hour&start=1711897200&end=1711983600&appid=4c90f7b876a2f86509856f59409ffc33&units=metric&lang=zh_cn